# Traductor

Pequeña prueba de concepto de implementación de un traductor de expresiones a lenguaje natural aprovechando el acceso a la estructura de árbol de las expresiones de **Calculate**.

In [ ]:
#pragma cling add_include_path("../calculate2.1.1rc6")
#include "calculate.hpp"

In [ ]:
#include <array>

In [ ]:
class Translator {
    using FuncMap = std::unordered_map<std::string, std::string>;
    using OperMap = std::unordered_map<std::string, std::array<std::string, 3>>;

    calculate::Parser _parser;
    FuncMap _function_aliases;
    OperMap _operator_aliases;
    std::regex _adapter;

    std::string _translate(
        const calculate::Parser::Expression& expression,
        bool verbose=false
    ) {
        decltype(_function_aliases.begin()) f;
        decltype(_operator_aliases.begin()) o;
        auto translation = std::string{};
        auto token = expression.token();
        auto n = expression.branches();

        switch (expression.type()) {
        case (calculate::Parser::SymbolType::FUNCTION):
            f = _function_aliases.find(token);
            if (f == _function_aliases.end())
                throw calculate::UndefinedSymbol(token);

            translation += f->second;
            for (std::size_t i = 0; i < n; i++) {
                translation += _translate(expression[i]);
                if (n > 1 && i < n - 1)
                    translation += (i == n - 2) ? "y " : ", ";
            }
            break;

        case (calculate::Parser::SymbolType::OPERATOR):
            o = _operator_aliases.find(token);
            if (o == _operator_aliases.end())
                throw calculate::UndefinedSymbol(token);

            if (verbose)
                translation += o->second[1];
            translation += _translate(expression[0]);
            translation += o->second[verbose ? 2 : 0];
            translation += _translate(expression[1], expression[1].branches() > 1);
            break;

        default:
            translation += token;
            translation += " ";
        }
        return translation;
    }

public:
    Translator() :
            _parser{},
            _function_aliases{},
            _operator_aliases{},
            _adapter{"(a el)"}
    {
        _function_aliases.insert({
            {"id", ""},
            {"neg", "menos "},
            {"inv", "la inversa de "},
            {"exp", "la exponencial de "},
            {"log", "el logaritmo de "},
            {"sin", "el seno de "},
            {"cos", "el coseno de "},
            {"tan", "la tangente de "}
        });

        _operator_aliases.insert({
            {"+", {"más ", "el resultado de sumar ", "y "}},
            {"-", {"menos ", "el resultado de restar ", "y "}},
            {"*", {"por ", "el resultado de multiplicar ", "por "}},
            {"/", {"entre ", "el resultado de dividir ", "entre "}},
            {"^", {"elevado a ", "el resultado de elevar ", "a "}}
        });
    }

    std::string translate(const std::string& expression) {
        auto translation = _translate(_parser.parse(expression));
        translation.pop_back();
        return std::regex_replace(translation, _adapter, "al");
    }
};

Translator translator;

In [ ]:
translator.translate("(1+2)*3")

In [ ]:
translator.translate("1+2*3")

In [ ]:
translator.translate("1+2^(3+4)")